In [ ]:
import pandas as pd 
import json

In [ ]:
import json
import ast

dict_list = []
count = 0
with open("data/review-New_York_10.json", "r", encoding="utf-8") as f:
    for line_num, line in enumerate(f, start=1):
        line = line.strip()  # remove leading/trailing whitespace
        if not line:
            continue  # skip empty lines
        try:
            # Try JSON parsing first (double quotes)
            obj = json.loads(line)
        except json.JSONDecodeError:
            try:
                # Fallback for Python dict strings (single quotes)
                obj = ast.literal_eval(line)
            except Exception:
                print(f"Skipping invalid line {line_num}: {line[:100]}...")
                continue
        dict_list.append(obj)
        count+=1
        if count > 100000:
            break

print(f"Total valid entries: {len(dict_list)}")



    

In [ ]:
df_reviews = pd.DataFrame(dict_list)

In [ ]:
df_reviews = df_reviews.dropna(subset=['text']).reset_index(drop=True)


In [ ]:
df_reviews.columns

In [ ]:
import json
import ast

dict_list = []

with open("data/meta-New_York.json", "r", encoding="utf-8") as f:
    for line_num, line in enumerate(f, start=1):
        line = line.strip()  # remove leading/trailing whitespace
        if not line:
            continue  # skip empty lines
        try:
            # Try JSON parsing first (double quotes)
            obj = json.loads(line)
        except json.JSONDecodeError:
            try:
                # Fallback for Python dict strings (single quotes)
                obj = ast.literal_eval(line)
            except Exception:
                print(f"Skipping invalid line {line_num}: {line[:100]}...")
                continue
        dict_list.append(obj)

print(f"Total valid entries: {len(dict_list)}")

In [ ]:
df_meta = pd.DataFrame(dict_list)

In [ ]:
merged_df = pd.merge(df_reviews, df_meta, on='gmap_id', how='inner')

In [ ]:
merged_df = merged_df.drop_duplicates(subset=['name_x','text','gmap_id'])

In [ ]:
merged_df.columns

In [ ]:
final_columns = ['text','resp','description','category','name_y', 'rating']
final_df = merged_df[final_columns]

for col in final_df.columns:
    final_df[col] = final_df[col].apply(
        lambda x: ", ".join(x) if isinstance(x, list) else x
    )

# Reset index so you get a new incrementing column
final_df = final_df.reset_index(drop=True)
final_df.index.name = "id"

final_df = final_df.rename(columns={
    "name_y": "Business Name",
    "category": "Category",
    "description": "Description",
    "text": "Review",
    "rating": "Rating",
    "resp": "Response"
})

final_df["user_message"] = (
    "Business Name: " + final_df["Business Name"].astype(str).fillna("N/A") + "\n"
    "Category: " + final_df["Category"].astype(str).fillna("N/A") + "\n"
    "Description: " + final_df["Description"].astype(str).fillna("N/A") + "\n"
    "Review: " + final_df["Review"].astype(str).fillna("N/A") + "\n"
    "Rating: " + final_df["Rating"].astype(str).fillna("N/A") + "\n"
    "Response: " + final_df["Response"].astype(str).fillna("N/A")
)



In [ ]:
final_df.to_csv("data.csv", index=True)  


In [ ]:
df = pd.read_csv("data.csv")


In [ ]:
print(df['user_message'][0])

In [ ]:
from openai import OpenAI
import json
import pandas as pd
import os
from dotenv import load_dotenv

client = OpenAI(
    api_key=os.environ.get("OPENAI_API_KEY"),
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/"
)


In [ ]:
from dotenv import load_dotenv
import os
load_dotenv(override=True)  # force overwrite existing environment variables

load_dotenv()  # loads .env variables
api_key = os.environ.get("OPENAI_API_KEY")
print("API key loaded:", api_key)  # just to check


In [ ]:
import os
print(os.getcwd())

In [ ]:
system_message = '''
 <task>
    Analyze Google reviews to gauge review quality by detecting problematic content.
  </task>
  
  <detection_criteria>
    <spam>Detect reviews that are fake, duplicate, or artificially generated</spam>
    <advertisement>Detect reviews that primarily promote other businesses or services</advertisement>
    <irrelevant_content>Detect reviews that don't relate to the actual business or location</irrelevant_content>
    <non_visitor_rant>Detect rants from users who likely never visited the location</non_visitor_rant>
    <toxicity>Detect reviews containing rudeness, racism, hate speech, harassment, or other toxic behavior</toxicity>
  </detection_criteria>
  
  <labeling_instructions>
    Label each detection category with 1 if the issue is present, 0 if not present.
  </labeling_instructions>
  
  <input_format>
    Review text will be provided for analysis.
  </input_format>
  
  <response_format>
    {
      "spam": 0 or 1,
      "advertisement": 0 or 1, 
      "irrelevant_content": 0 or 1,
      "non_visitor_rant": 0 or 1,
      "toxicity": 0 or 1
    }
  </response_format>
  
  <example>
    Input: "This place is terrible! I heard from my friend it's dirty and overpriced. Never going there!"
    Output: {"spam": 0, "advertisement": 0, "irrelevant_content": 0, "non_visitor_rant": 1, "toxicity": 0}
  </example>
  <example>
    Input: "Had dinner here last night with my family. The food was okay I guess, service was a bit slow. But speaking of great service, I have to mention that 
    I recently had my car detailed by Elite Auto Spa on Main Street and it was absolutely incredible! They made my 10-year-old Honda look brand new. 
    The owner Dave is super friendly and they use eco-friendly products. Only $89 for a full detail - such a steal! If anyone needs their car cleaned, 
    definitely check out Elite Auto Spa. They're open 7 days a week and take walk-ins. Tell them Mike sent you for 10% off your first visit!
    Anyway, the restaurant was fine. Might come back sometime."
    Output:  {"spam": 0, "advertisement": 1, "irrelevant_content": 1, "non_visitor_rant": 0, "toxicity": 0}
  </example>
  <example>
    Input: "This place is absolutely DISGUSTING and the staff are complete idiots who don't know anything about customer service!!! The manager Karen is a total b**** who needs to be fired immediately. Food gave me food poisoning and I'm going to sue them for everything they're worth.
    EVERYONE NEEDS TO BOYCOTT THIS TERRIBLE RESTAURANT. They are probably money laundering and breaking health codes. I saw mexican ROACHES in the kitchen (I didn't but whatever).
    By the way, if you want REAL Italian food, go to Tony's Pizza Palace downtown - they have the BEST pizza in the city and Maria the owner is amazing! Use code FOODIE20 for 20% off! They're having a grand opening special this week!
    Also check out my food blog at totally-real-food-reviews.net where I expose restaurants like this one. Follow me on Instagram @foodcritic_real for more honest reviews!
    DO NOT EAT HERE - YOU HAVE BEEN WARNED!!!"
    Output: {"spam": 0, "advertisement": 0, "irrelevant_content": 1, "non_visitor_rant": 0, "toxicity": 1}
  </example>  
'''

In [ ]:
from pydantic import BaseModel
from typing import List
import json
import os
import pandas as pd
from openai import OpenAI


class MessageInfo(BaseModel):
    spam: str
    advertisement: str
    irrelevant_content: str
    non_visitor_rant: str
    toxicity:str 

class Messages(BaseModel):
    messages: List[MessageInfo]

class BatchProcess():
    def __init__(self, df:pd.DataFrame, start_index: int, batch_start_number:int, system_message:str, client:OpenAI):
        """ 
        prepares raw jd from df, sends it to llms and writes outputs to json
        
        """
        self.df = df
        self.start_index = start_index #index of the dataframe to START
        self.batch_start_number = batch_start_number #batch number to START
        self.system_message = system_message
        self.client = client

    def process(self, no_of_batches:int, batch_size:int) -> None:
        #prepare the messages in the batch
        for i in range(no_of_batches):
            print(f"Processing batch {i}...")
            
            messages, row_ids, checkpoint = self.prepare_llm_inputs(self.df, self.start_index, batch_size) #process before llm
            llm_structured_output = self.get_structured_output_from_llm(self.system_message, messages) #pass to llm, get responses

            print("Processing row_ids...", row_ids)

            if len(llm_structured_output.messages) != len(row_ids):
                print(messages)
                print(llm_structured_output.messages)
                raise ValueError(f"Error: Expected batch size of {batch_size}, but got jobids: {len(row_ids)} and llmoutputs: {len(llm_structured_output.messages)}.")

            batch_jds = self.append_jb_ids(row_ids, llm_structured_output) #process output, append jd ids to the response

            self.write_to_file(batch_jds, checkpoint, self.batch_start_number) #write to json for the batch
            
            self.start_index = checkpoint

            self.batch_start_number += 1
        


       
    # need to iterate through the rows in the df for that batch from the start checkpoint, 
    def prepare_llm_inputs(self, df: pd.DataFrame, checkpoint:int, batch_size:int) -> tuple[str, List[int], int]:
        """ 
        This function iterates through the rows in the dataframe in that batch and prepares multiple raw JDs into one message/string for the llm
        returns the updated checkpoint, input prompt which is a string and the list of their job ids for addition later
        """
        messages =""
        job_ids = []
        counter = 1
        for i in range(checkpoint, checkpoint+batch_size):
            job_id = int(df.iloc[i]['id'])
            raw_jd = df.iloc[i]['user_message'].strip()
            job_ids.append(job_id)
            messages += f"<Job {counter}>\n{raw_jd}\n</Job {counter}>\n\n"
            counter+=1
        # Check if the number of job ids does not match the batch size
        if len(job_ids) != batch_size:
            
            raise ValueError(f"Error: Expected batch size of {batch_size}, but got {len(job_ids)}.")

        return (messages, job_ids, checkpoint + batch_size)
    

            
    def get_structured_output_from_llm(self, system_message: str, user_message:str) -> Messages:
        """ 
        sends the messages to gemini, returns a Messages object
        """
        response = self.client.beta.chat.completions.parse(
            model="gemini-2.0-flash-lite",
            messages=[
                {"role": "system", "content": system_message},
                {
                    "role": "user",
                    "content": user_message
                }
            ],
            response_format=Messages,
        )
        return response.choices[0].message.parsed
    
    # we need to append the job ids to each message, saves output to json file
    def append_jb_ids(self, job_ids:List, messages_list: Messages) -> List:
        batch_jds = []
        for idx, message in enumerate(messages_list.messages):
            temp = message.model_dump()
            #print(temp, idx)
            temp['job_id'] = job_ids[idx]
            batch_jds.append(temp)

        return batch_jds
    
    def write_to_file(self, batch_jds:List, row_checkpoint, batch_no) -> None:
            row_checkpoint -= 1
            dir_name = "extracted_labels"
            os.makedirs(dir_name, exist_ok=True)

            filename = f"{dir_name}/batch_{batch_no}_row_{row_checkpoint}_extracted_labels.json"

            with open(filename, "w") as f:
                json.dump(batch_jds, f, indent=4) 


In [ ]:
shuffled_df = df.sample(frac=1, random_state=42).reset_index(drop=True)



In [ ]:
batch = BatchProcess(shuffled_df, 9700, 485, system_message, client)

In [ ]:
batch.process(500, 20)

In [ ]:
messages, job_ids, checkpoint = batch.prepare_llm_inputs(fake_df, 0, 2)

In [ ]:
response = batch.get_structured_output_from_llm(system_message, messages)

In [ ]:
response

In [ ]:
print(df.head(10).iloc[9]['user_message'])

In [ ]:
import glob
# get all json files in the folder
json_files = glob.glob("extracted_labels/*.json")

combined_data = []

for file in json_files:
    with open(file, "r", encoding="utf-8") as f:
        data = json.load(f)
        if isinstance(data, list):
            combined_data.extend(data)   # merge lists
        else:
            combined_data.append(data)   # append objects

# save into one file
with open("combined.json", "w", encoding="utf-8") as f:
    json.dump(combined_data, f, indent=2, ensure_ascii=False)

In [ ]:
len(combined_data)

In [ ]:
combined_data_df = pd.DataFrame(combined_data)
combined_data_df = combined_data_df.rename(columns={"job_id": "id"})
labeled_final = pd.merge(combined_data_df, shuffled_df, on="id", how="left")

labeled_final.to_csv("final_labeled.csv", index=False, encoding="utf-8")
